In [21]:
import requests
from PIL import Image, ImageDraw
#!pip install CMake
#!pip install face_recognition
import face_recognition
import numpy as np
import sys
sys.path.append(r'/home/jupyter/reusable_code')
import google_api_functions as gaf
from google.cloud import bigquery # To run BQ statements
creds=gaf.Authenticate_Google(r'/home/jupyter/reusable_code/') # Return logged-in credentials


In [22]:
bq = bigquery.Client(project='itv-bde-analytics-dev',credentials=creds)
query='''select * FROM `itv-bde-svod-prd.svod_ccm.images` where img_name='16x9' '''
df = bq.query(query).to_dataframe()
url_list=df['img_url'].values.tolist()
url_list=[i for i in url_list if i.split('/')[4]=='episode']
url_list[:10]

In [23]:
len(url_list)

In [32]:
bblogo=Image.open(requests.get('https://www.trustedreviews.com/wp-content/uploads/sites/54/2019/11/britbox-logo-920x380.png',stream=True).raw)
url_list=['https://imgix.lifehacker.com.au/content/uploads/sites/4/2020/11/24/doctor-who-britbox-1.jpg?ar=16%3A9&auto=format&fit=crop&q=65&w=1280',\
         'https://imgix.pedestrian.tv/content/uploads/2020/11/23/Screen-Shot-2020-11-23-at-11.jpg?ar=16%3A9&auto=format&crop=focal&fit=crop&q=65&w=1200',
         'https://images-na.ssl-images-amazon.com/images/S/sgp-catalog-images/region_US/BritBox-10085863-Full-Image_GalleryCover-en-US-1596530360751._UR1920,1080_RI_.jpg']


for n,url in enumerate([url_list[0]]):
    # Get picture from url
    pic=requests.get(url,stream=True)
    
    # Load into memory
    im = Image.open(pic.raw)
    
    # Retrieve image dimensions
    width, height = im.size
    if width/height==16/9:
        new_width=width*0.75 # Calculate the new ratio to make a 4:3 (or rather a 12:9)
        
        
        ################################################################################################################################
        # Crop centrally
        ################################################################################################################################
        # Define the rectangle to crop
        top,bottom=(0,height)
        left=(width-new_width)/2
        right=left+new_width
        # Crop it
        im1 = im.crop((left, top, right, bottom)) 
  

        # Add the BB logo
    
        
    
        # Shows the image in image viewer 
        # im1.show()
        
        # Check it's right
        #new_width, new_height=im1.size
        #if new_width/new_height==4/3:
        #    print('Now it''s a 4:3')
        
        # Save--> this could be fleshed out to retrieve existing name and format 
        im1.save('Cropped_image{}.jpg'.format(n))
        
        ################################################################################################################################
        # Crop based on faces
        ################################################################################################################################
        
        im_array=np.asarray(im)
        face_locations = face_recognition.face_locations(im_array,number_of_times_to_upsample=1,model='hog')
        
        # Demonstrate the detected faces
        #for face_location in face_locations:
            # Print the location of each face in this image. Each face is a list of co-ordinates in (top, right, bottom, left) order.
        #    facetop, faceright, facebottom, faceleft = face_location
            # Let's draw a box around the face
        #    draw = ImageDraw.Draw(im)
         #   draw.rectangle((faceleft,facetop,faceright,facebottom))
           
        # Calculate the areas of the pic bounded by the faces and centralise them to crop
        minleft = max(min([face_location[3] for face_location in face_locations]),0)
        maxright= min(max([face_location[1] for face_location in face_locations]),width)
        if maxright-minleft<=new_width:
            newleft=(minleft+maxright)/2-new_width/2
            newright=newleft+new_width
            im2 = im.crop((newleft, top, newright, bottom)) 
            im2.save('Cropped_image{}- Face Centred.jpg'.format(n))
        
        im.save('Cropped_image{}- Original 16x9.jpg'.format(n))
        #im.show()
        #print(face_locations)
        # Empty memory
        im1.close()
        im.close()
        
    else:
        print('Not a 16:9')
        
    
    print(width, height)

In [24]:
bblogo=Image.open(requests.get('https://www.trustedreviews.com/wp-content/uploads/sites/54/2019/11/britbox-logo-920x380.png',stream=True).raw)


In [34]:
help(bblogo.paste)